# Example: Import district heating network from GeoJSON files
============================================================

This example demonstrates how to import a district heating network
from three GeoJSON files representing network topology, buildings,
and supply points.


In [1]:

import os
from uesgraphs import UESGraph
from uesgraphs.examples import e1_readme_example as e1

#Set workspace
workspace = e1.workspace_example("e15")


# Get path to example data
uesgraphs_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(uesgraphs_dir, 'data','examples', 'e15_geojson')

# Create graph and import GeoJSON
graph = UESGraph()
graph.from_geojson(
    network_path=os.path.join(data_dir, 'network.geojson'),
    buildings_path=os.path.join(data_dir, 'buildings.geojson'),
    supply_path=os.path.join(data_dir, 'supply.geojson'),
    name='simple_district',
    save_path=workspace,
    generate_visualizations=True
)


print(f"============ Uesgraphs Import from GeoJSON successful! ============")
print(f"Network nodes: {graph.number_of_nodes('heating')}")
print(f"Buildings: {graph.number_of_nodes('building')}")
print(f"Total length: {graph.network_length:.2f} m")
print(f"Visualizations of network generation can be found in: {os.path.join(workspace, 'visualizations')}")

graph.to_json(os.path.join(workspace, 'simple_district_graph.json'))

Logfile findable here: C:\Users\rka-lko\AppData\Local\Temp\6\Visuals_20251030_145014.log


Ignoring fixed y limits to fulfill fixed data aspect with adjustable data limits.
Ignoring fixed y limits to fulfill fixed data aspect with adjustable data limits.


Saved visualization to E:\rka_lko\git\uesgraphs\workspace\e15\visuals_of_uesgraph_creation\1_basic_uesgraph.pdf
Logfile findable here: C:\Users\rka-lko\AppData\Local\Temp\6\Visuals_20251030_145014.log


Ignoring fixed y limits to fulfill fixed data aspect with adjustable data limits.


Saved visualization to E:\rka_lko\git\uesgraphs\workspace\e15\visuals_of_uesgraph_creation\2_with_supply.pdf
Logfile findable here: C:\Users\rka-lko\AppData\Local\Temp\6\Visuals_20251030_145014.log
Saved visualization to E:\rka_lko\git\uesgraphs\workspace\e15\visuals_of_uesgraph_creation\3_with_bldg.pdf


Ignoring fixed y limits to fulfill fixed data aspect with adjustable data limits.


Logfile findable here: C:\Users\rka-lko\AppData\Local\Temp\6\Visuals_20251030_145014.log
Saved visualization to E:\rka_lko\git\uesgraphs\workspace\e15\visuals_of_uesgraph_creation\4_coords_transformed.pdf
Total network length (m):  426.27
============ Uesgraphs Import from GeoJSON successful! ============
Network nodes: 8
Buildings: 5
Total length: 426.27 m
Visualizations of network generation can be found in: E:\rka_lko\git\uesgraphs\workspace\e15\visualizations


'E:\\rka_lko\\git\\uesgraphs\\workspace\\e15\\simple_district_graph.json\\nodes.json'

In [2]:
graph.nodes

NodeView((1002, 1004, 1006, 1007, 1010, 1014, 1018, 1022, 1025, 1026, 1027, 1028, 1029))

In [3]:

import pandas as pd
import os
from pathlib import Path
import logging


def load_component_parameters(excel_path, component_type):
    """
    Load component parameters from an Excel file.
    
    Reads a specific sheet from an Excel file and returns parameters as a dictionary.
    Expected Excel structure:
    - Column A: Parameter (parameter names)
    - Column B: Value (parameter values)
    
    Parameters
    ----------
    excel_path : str or Path
        Path to the Excel file containing component parameters
    component_type : str
        Type of component, must be one of: 'pipes', 'supply', 'demands', 'simulation'
        This determines which sheet to read from the Excel file
        
    Returns
    -------
    dict
        Dictionary with parameter names as keys and their values
        Returns empty dict if sheet not found
        
    Raises
    ------
    FileNotFoundError
        If the Excel file does not exist
    ValueError
        If the component_type is not valid or Excel structure is incorrect
        
    Examples
    --------
    >>> params = load_component_parameters('parameters.xlsx', 'pipes')
    >>> print(params['dp_nominal'])
    0.10
    """
    # Validate component type
    valid_types = ['Pipes', 'Supply', 'Demands', 'Simulation']
    if component_type not in valid_types:
        raise ValueError(
            f"Invalid component_type '{component_type}'. "
            f"Must be one of: {', '.join(valid_types)}"
        )
    
    # Check if file exists
    if not os.path.exists(excel_path):
        raise FileNotFoundError(f"Excel file not found: {excel_path}")
    
    try:
        # Read the specific sheet
        df = pd.read_excel(excel_path, sheet_name=component_type)
        
        # Validate Excel structure
        if len(df.columns) < 2:
            raise ValueError(
                f"Excel sheet '{component_type}' must have at least 2 columns "
                "(Parameter and Value)"
            )
        
        # Check if first row contains expected column names
        if 'Parameter' not in df.columns and 'parameter' not in df.columns.str.lower():
            # Assume first two columns are Parameter and Value
            df.columns = ['Parameter', 'Value'] + list(df.columns[2:])
        
        # Create dictionary from Parameter and Value columns
        # Drop rows where Parameter is NaN
        df_clean = df[['Parameter', 'Value']].dropna(subset=['Parameter'])
        
        # Convert to dictionary
        param_dict = dict(zip(df_clean['Parameter'], df_clean['Value']))
        
        return param_dict
        
    except ValueError as e:
        if "Worksheet named" in str(e):
            # Sheet doesn't exist
            logging.warning(
                f"Sheet '{component_type}' not found in {excel_path}. "
                f"Returning empty dictionary."
            )
            return {}
        else:
            raise
    
    except Exception as e:
        raise Exception(
            f"Error reading Excel file {excel_path}, sheet '{component_type}': {e}"
        )

In [4]:
load_component_parameters(r"C:\Users\rka-lko\Downloads\uesgraphs_parameters_template.xlsx", "Pipes")

{'template': 'PlugFlowPipeEmbedded',
 'template_path': 'E:\\rka_lko\\git\\uesgraphs\\uesgraphs\\data\\templates\\network\\pipe\\AixLib_Fluid_DistrictHeatingCooling_Pipes_PlugFlowPipeEmbedded.mako',
 'dIns': 0.04,
 'kIns': 0.03,
 'fac': 1.5,
 'roughness': 0,
 'ground_depth': 1,
 'static_pipe_length': nan,
 'allowFlowReversal': 'TRUE'}

In [5]:
from mako.template import Template
def parse_template_parameters(template_path, logger=None):
    """
    Parse a Mako template file to extract required and optional parameters.
    
    Reads a .mako template file and extracts the parameter lists defined in:
    - get_main_parameters: Required parameters (must be provided)
    - get_aux_parameters: Auxiliary/optional parameters (will use defaults if not provided)
    
    Parameters
    ----------
    template_path : str or Path
        Path to the Mako template file (.mako)
    logger : logging.Logger, optional
        Logger for status messages
        
    Returns
    -------
    tuple (list, list)
        (main_parameters, aux_parameters)
        - main_parameters: List of required parameter names
        - aux_parameters: List of optional parameter names
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    AttributeError
        If template doesn't define get_main_parameters or get_aux_parameters
    Exception
        For other template parsing errors
        
    Examples
    --------
    >>> main, aux = parse_template_parameters('pipe_template.mako')
    >>> print(main)
    ['length', 'diameter']
    >>> print(aux)
    ['m_flow_nominal', 'dp_nominal', 'roughness']
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    # Check if template file exists
    if not os.path.exists(template_path):
        error_msg = f"Template file not found: {template_path}"
        logger.error(error_msg)
        raise FileNotFoundError(error_msg)
    
    try:
        # Load the Mako template
        logger.debug(f"Loading template: {template_path}")
        template = Template(filename=str(template_path))
        
        # Extract main (required) parameters
        try:
            main_params_str = template.get_def("get_main_parameters").render()
            main_params = main_params_str.split()
            logger.debug(f"Main parameters found: {main_params}")
        except AttributeError:
            logger.warning(
                f"Template {template_path} does not define 'get_main_parameters'. "
                "Assuming no required parameters."
            )
            main_params = []
        
        # Extract auxiliary (optional) parameters
        try:
            aux_params_str = template.get_def("get_aux_parameters").render()
            aux_params = aux_params_str.split()
            logger.debug(f"Auxiliary parameters found: {aux_params}")
        except AttributeError:
            logger.warning(
                f"Template {template_path} does not define 'get_aux_parameters'. "
                "Assuming no optional parameters."
            )
            aux_params = []
        
        logger.info(
            f"Template parsed successfully: "
            f"{len(main_params)} main params, {len(aux_params)} aux params"
        )
        
        return main_params, aux_params
        
    except Exception as e:
        error_msg = f"Error parsing template {template_path}: {e}"
        logger.error(error_msg)
        raise Exception(error_msg)

In [6]:
parse_template_parameters(r"E:\rka_lko\git\uesgraphs\uesgraphs\data\templates\network\pipe\AixLib_Fluid_DistrictHeatingCooling_Pipes_PlugFlowPipeEmbedded.mako")

(['length', 'dIns', 'kIns'],
 ['CPip',
  'VEqu',
  'sta_default',
  'cp_default',
  'C',
  'rho_default',
  'energyDynamics',
  'use_zeta',
  'from_dp',
  'dh',
  'v_nominal',
  'ReC',
  'roughness',
  'm_flow_nominal',
  'm_flow_small',
  'cPip',
  'rhoPip',
  'thickness',
  'T_start_in',
  'T_start_out',
  'initDelay',
  'm_flow_start',
  'R',
  'fac',
  'sum_zetas',
  'linearized',
  'rho_soi',
  'c',
  'thickness_soi',
  'lambda_ground',
  'd_in',
  'nParallel',
  'T0',
  '_m_flow_start',
  '_dp_start',
  'show_T',
  'allowFlowReversal'])

In [7]:

def assign_pipe_parameters(uesgraph, template_path, excel_path=None, logger=None):
    """
    Assign parameters to pipe edges in the uesgraph according to the flow chart logic.
    
    This function follows the validation flow:
    1. Load and parse the template file
    2. Extract MAIN (required) and AUX (optional) parameters
    3. For each edge individually:
       - Check MAIN parameters: in edge → keep, not in edge → try Excel, missing → ERROR
       - Check AUX parameters: in edge → keep, not in edge → try Excel, missing → WARNING
    4. Apply parameters from Excel where needed (never overwrite existing edge attributes)
    
    Parameters
    ----------
    uesgraph : UESGraph
        The urban energy system graph object
    template_path : str or Path
        Path to the pipe template file (.mako)
    excel_path : str or Path, optional
        Path to Excel file containing component parameters
        If None, only graph attributes are used
    logger : logging.Logger, optional
        Logger for status messages and warnings
        
    Returns
    -------
    uesgraph : UESGraph
        Updated graph with assigned parameters
    validation_report : dict
        Report containing:
        - 'success': bool, whether all MAIN parameters were found for all edges
        - 'edges_validated': int, number of edges processed
        - 'params_from_graph': int, parameters already in graph
        - 'params_from_excel': int, parameters added from Excel
        - 'missing_main': list of (edge, param) tuples for missing required parameters
        - 'missing_aux': set of parameter names missing across all edges
        - 'errors': list of error messages
        - 'warnings': list of warning messages
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    ValueError
        If required MAIN parameters are missing for any edge
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    validation_report = {
        'success': True,
        'edges_validated': 0,
        'params_from_graph': 0,
        'params_from_excel': 0,
        'missing_main': [],
        'missing_aux': set(),
        'errors': [],
        'warnings': []
    }
    
    # Step 1: Load template file
    logger.info(f"Loading template file: {template_path}")
    if not os.path.exists(template_path):
        error_msg = f"Template file not found: {template_path}"
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        validation_report['success'] = False
        raise FileNotFoundError(error_msg)
    
    # Step 2: Parse template and extract parameters
    logger.info("Parsing template to extract parameter requirements")
    try:
        main_parameters, aux_parameters = parse_template_parameters(template_path, logger)
    except Exception as e:
        error_msg = f"Failed to parse template: {e}"
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        validation_report['success'] = False
        raise
    
    logger.info(f"Template requires {len(main_parameters)} MAIN and {len(aux_parameters)} AUX parameters")
    logger.debug(f"MAIN: {main_parameters}")
    logger.debug(f"AUX: {aux_parameters}")
    
    # Step 3: Load Excel parameters if provided
    excel_params = {}
    if excel_path is not None:
        try:
            logger.info(f"Loading parameters from Excel: {excel_path}")
            excel_params = load_component_parameters(excel_path, 'Pipes')
            logger.debug(f"Excel parameters loaded: {list(excel_params.keys())}")
        except Exception as e:
            warning_msg = f"Could not load Excel parameters: {e}"
            logger.warning(warning_msg)
            validation_report['warnings'].append(warning_msg)
    
    # Step 4: Process each edge individually
    total_edges = len(list(uesgraph.edges()))
    logger.info(f"Processing {total_edges} edges...")
    
    for edge_idx, edge in enumerate(uesgraph.edges(), 1):
        edge_data = uesgraph.edges[edge]
        logger.debug(f"Processing edge {edge_idx}/{total_edges}: {edge}")
        
        # Step 4a: Check and apply MAIN parameters for this edge
        for param in main_parameters:
            if param in edge_data:
                # Parameter already in graph - keep it
                validation_report['params_from_graph'] += 1
                logger.debug(f"  ✓ MAIN '{param}' found in edge")
            elif param in excel_params:
                # Parameter not in edge, but available in Excel - apply it
                edge_data[param] = excel_params[param]
                validation_report['params_from_excel'] += 1
                logger.debug(f"  ✓ MAIN '{param}' applied from Excel")
            else:
                # Parameter missing - ERROR!
                validation_report['missing_main'].append((edge, param))
                validation_report['success'] = False
                error_msg = f"Edge {edge}: Missing MAIN parameter '{param}'"
                logger.error(error_msg)
                validation_report['errors'].append(error_msg)
        
        # Step 4b: Check and apply AUX parameters for this edge
        for param in aux_parameters:
            if param in edge_data:
                # Parameter already in graph - keep it
                validation_report['params_from_graph'] += 1
                logger.debug(f"  ✓ AUX '{param}' found in edge")
            elif param in excel_params:
                # Parameter not in edge, but available in Excel - apply it
                edge_data[param] = excel_params[param]
                validation_report['params_from_excel'] += 1
                logger.debug(f"  ✓ AUX '{param}' applied from Excel")
            else:
                # Parameter missing - will use Modelica default (collect for summary)
                validation_report['missing_aux'].add(param)
                logger.debug(f"  ⚠ AUX '{param}' not provided - will use Modelica default")
        
        validation_report['edges_validated'] += 1
    
    # Step 5: Report summary
    logger.info(f"✓ Processed {validation_report['edges_validated']} edges")
    logger.info(f"  - Parameters from graph: {validation_report['params_from_graph']}")
    logger.info(f"  - Parameters from Excel: {validation_report['params_from_excel']}")
    
    # Summary of missing AUX parameters (if any)
    if validation_report['missing_aux']:
        missing_count = len(validation_report['missing_aux'])
        summary_msg = (
            f"⚠ {missing_count} AUX parameter(s) not provided, will use Modelica defaults:\n"
            f"   {', '.join(sorted(validation_report['missing_aux']))}"
        )
        logger.warning(summary_msg)
        validation_report['warnings'].append(summary_msg)
    
    # Check if validation was successful
    if not validation_report['success']:
        error_count = len(validation_report['missing_main'])
        error_msg = (
            f"Validation FAILED: {error_count} missing MAIN parameter(s)\n"
            f"Missing parameters per edge:\n"
        )
        for edge, param in validation_report['missing_main']:
            error_msg += f"  - Edge {edge}: '{param}'\n"
        error_msg += (
            f"\nFix suggestions:\n"
            f"  → If parameter varies per edge: add to uesgraph edge attributes\n"
            f"  → If parameter is same for all edges: add to Excel sheet 'pipes'"
        )
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        raise ValueError(error_msg)
    else:
        logger.info("✓ All MAIN parameters successfully validated and applied")
    
    return uesgraph, validation_report

In [23]:
graph2 = assign_pipe_parameters(graph, r"E:\rka_lko\git\uesgraphs\uesgraphs\data\templates\network\pipe\AixLib_Fluid_DistrictHeatingCooling_Pipes_PlugFlowPipeEmbedded.mako", r"C:\Users\rka-lko\Downloads\uesgraphs_parameters_template.xlsx")

34 AUX parameter(s) not provided for edges, will use Modelica defaults: C, CPip, R, ReC, T0, T_start_in, T_start_out, VEqu, _dp_start, _m_flow_start, c, cPip, cp_default, d_in, dh, energyDynamics, from_dp, initDelay, lambda_ground, linearized, m_flow_nominal, m_flow_small, m_flow_start, nParallel, rhoPip, rho_default, rho_soi, show_T, sta_default, sum_zetas, thickness, thickness_soi, use_zeta, v_nominal
C:\Users\rka-lko\AppData\Local\Temp\6\ipykernel_16216\1232783304.py:278: UserWarning: 34 AUX parameter(s) not provided for edges, will use Modelica defaults: C, CPip, R, ReC, T0, T_start_in, T_start_out, VEqu, _dp_start, _m_flow_start, c, cPip, cp_default, d_in, dh, energyDynamics, from_dp, initDelay, lambda_ground, linearized, m_flow_nominal, m_flow_small, m_flow_start, nParallel, rhoPip, rho_default, rho_soi, show_T, sta_default, sum_zetas, thickness, thickness_soi, use_zeta, v_nominal
  warnings.warn(warning_msg, UserWarning)


In [9]:
graph.edges(data=True)

EdgeDataView([(1002, 1010, {'attr_dict': {'id': 9, 'DN': 65, 'length': 61.98113731669513}, 'diameter': 0.0697, 'dIns': 0.1, 'length': 62.67448664094058, 'kIns': 0.03, 'roughness': 0, 'fac': 1.5, 'allowFlowReversal': 'TRUE'}), (1002, 1014, {'attr_dict': {'id': 13, 'DN': 200, 'length': 40.10060652921126}, 'diameter': 0.2101, 'dIns': 0.1, 'length': 40.10060652984215, 'kIns': 0.03, 'roughness': 0, 'fac': 1.5, 'allowFlowReversal': 'TRUE'}), (1002, 1025, {'attr_dict': {'id': 2, 'DN': 200, 'length': 23.793988913605595}, 'diameter': 0.2101, 'length': 23.793988913832845, 'dIns': 0.04, 'kIns': 0.03, 'roughness': 0, 'fac': 1.5, 'allowFlowReversal': 'TRUE'}), (1004, 1006, {'attr_dict': {'id': 5, 'DN': 125, 'length': 23.152279454397373}, 'diameter': 0.1325, 'dIns': 0.1, 'length': 23.15227945754485, 'kIns': 0.03, 'roughness': 0, 'fac': 1.5, 'allowFlowReversal': 'TRUE'}), (1004, 1007, {'attr_dict': {'id': 7, 'DN': 125, 'length': 50.66733186046747}, 'diameter': 0.1325, 'dIns': 0.1, 'length': 67.213436

In [10]:

def assign_supply_parameters(uesgraph, template_path, excel_path=None, logger=None):
    """
    Assign parameters to supply nodes in the uesgraph according to the flow chart logic.
    
    This function follows the validation flow:
    1. Load and parse the template file
    2. Extract MAIN (required) and AUX (optional) parameters
    3. For each supply node individually:
       - Check MAIN parameters: in node → keep, not in node → try Excel, missing → ERROR
       - Check AUX parameters: in node → keep, not in node → try Excel, missing → WARNING
    4. Apply parameters from Excel where needed (never overwrite existing node attributes)
    
    Parameters
    ----------
    uesgraph : UESGraph
        The urban energy system graph object
    template_path : str or Path
        Path to the supply template file (.mako)
    excel_path : str or Path, optional
        Path to Excel file containing component parameters
        If None, only graph attributes are used
    logger : logging.Logger, optional
        Logger for status messages and warnings
        
    Returns
    -------
    uesgraph : UESGraph
        Updated graph with assigned parameters
    validation_report : dict
        Report containing:
        - 'success': bool, whether all MAIN parameters were found for all nodes
        - 'nodes_validated': int, number of supply nodes processed
        - 'params_from_graph': int, parameters already in graph
        - 'params_from_excel': int, parameters added from Excel
        - 'missing_main': list of (node, param) tuples for missing required parameters
        - 'missing_aux': set of parameter names missing across all nodes
        - 'errors': list of error messages
        - 'warnings': list of warning messages
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    ValueError
        If required MAIN parameters are missing for any supply node
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    validation_report = {
        'success': True,
        'nodes_validated': 0,
        'params_from_graph': 0,
        'params_from_excel': 0,
        'missing_main': [],
        'missing_aux': set(),
        'errors': [],
        'warnings': []
    }
    
    # Step 1: Load template file
    logger.info(f"Loading template file: {template_path}")
    if not os.path.exists(template_path):
        error_msg = f"Template file not found: {template_path}"
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        validation_report['success'] = False
        raise FileNotFoundError(error_msg)
    
    # Step 2: Parse template and extract parameters
    logger.info("Parsing template to extract parameter requirements")
    try:
        main_parameters, aux_parameters = parse_template_parameters(template_path, logger)
    except Exception as e:
        error_msg = f"Failed to parse template: {e}"
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        validation_report['success'] = False
        raise
    
    logger.info(f"Template requires {len(main_parameters)} MAIN and {len(aux_parameters)} AUX parameters")
    logger.debug(f"MAIN: {main_parameters}")
    logger.debug(f"AUX: {aux_parameters}")
    
    # Step 3: Load Excel parameters if provided
    excel_params = {}
    if excel_path is not None:
        try:
            logger.info(f"Loading parameters from Excel: {excel_path}")
            excel_params = load_component_parameters(excel_path, 'Supply')
            logger.debug(f"Excel parameters loaded: {list(excel_params.keys())}")
        except Exception as e:
            warning_msg = f"Could not load Excel parameters: {e}"
            logger.warning(warning_msg)
            validation_report['warnings'].append(warning_msg)
    
    # Step 4: Find supply nodes
    network_type = uesgraph.graph.get("network_type", "heating")
    is_supply_key = f"is_supply_{network_type}"
    
    supply_nodes = []
    for node in uesgraph.nodelist_building:
        if uesgraph.nodes[node].get(is_supply_key, False):
            supply_nodes.append(node)
    
    if not supply_nodes:
        warning_msg = f"No supply nodes found (looking for '{is_supply_key}' = True)"
        logger.warning(warning_msg)
        validation_report['warnings'].append(warning_msg)
        return uesgraph, validation_report
    
    # Step 5: Process each supply node individually
    total_nodes = len(supply_nodes)
    logger.info(f"Processing {total_nodes} supply node(s)...")
    
    for node_idx, node in enumerate(supply_nodes, 1):
        node_data = uesgraph.nodes[node]
        logger.debug(f"Processing supply node {node_idx}/{total_nodes}: {node}")
        
        # Step 5a: Check and apply MAIN parameters for this node
        for param in main_parameters:
            if param in node_data:
                # Parameter already in graph - keep it
                validation_report['params_from_graph'] += 1
                logger.debug(f"  ✓ MAIN '{param}' found in node")
            elif param in excel_params:
                # Parameter not in node, but available in Excel - apply it
                node_data[param] = excel_params[param]
                validation_report['params_from_excel'] += 1
                logger.debug(f"  ✓ MAIN '{param}' applied from Excel")
            else:
                # Parameter missing - ERROR!
                validation_report['missing_main'].append((node, param))
                validation_report['success'] = False
                error_msg = f"Node {node}: Missing MAIN parameter '{param}'"
                logger.error(error_msg)
                validation_report['errors'].append(error_msg)
        
        # Step 5b: Check and apply AUX parameters for this node
        for param in aux_parameters:
            if param in node_data:
                # Parameter already in graph - keep it
                validation_report['params_from_graph'] += 1
                logger.debug(f"  ✓ AUX '{param}' found in node")
            elif param in excel_params:
                # Parameter not in node, but available in Excel - apply it
                node_data[param] = excel_params[param]
                validation_report['params_from_excel'] += 1
                logger.debug(f"  ✓ AUX '{param}' applied from Excel")
            else:
                # Parameter missing - will use Modelica default (collect for summary)
                validation_report['missing_aux'].add(param)
                logger.debug(f"  ⚠ AUX '{param}' not provided - will use Modelica default")
        
        validation_report['nodes_validated'] += 1
    
    # Step 6: Report summary
    logger.info(f"✓ Processed {validation_report['nodes_validated']} supply node(s)")
    logger.info(f"  - Parameters from graph: {validation_report['params_from_graph']}")
    logger.info(f"  - Parameters from Excel: {validation_report['params_from_excel']}")
    
    # Summary of missing AUX parameters (if any)
    if validation_report['missing_aux']:
        missing_count = len(validation_report['missing_aux'])
        summary_msg = (
            f"⚠ {missing_count} AUX parameter(s) not provided, will use Modelica defaults:\n"
            f"   {', '.join(sorted(validation_report['missing_aux']))}"
        )
        logger.warning(summary_msg)
        validation_report['warnings'].append(summary_msg)
    
    # Check if validation was successful
    if not validation_report['success']:
        error_count = len(validation_report['missing_main'])
        error_msg = (
            f"Validation FAILED: {error_count} missing MAIN parameter(s)\n"
            f"Missing parameters per node:\n"
        )
        for node, param in validation_report['missing_main']:
            error_msg += f"  - Node {node}: '{param}'\n"
        error_msg += (
            f"\nFix suggestions:\n"
            f"  → If parameter varies per node: add to uesgraph node attributes\n"
            f"  → If parameter is same for all supply nodes: add to Excel sheet 'Supply'"
        )
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        raise ValueError(error_msg)
    else:
        logger.info("✓ All MAIN parameters successfully validated and applied")
    
    return uesgraph, validation_report

In [22]:
graph2 = assign_supply_parameters(graph2, r"E:\rka_lko\git\uesgraphs\uesgraphs\data\templates\network\supply\AixLib_Fluid_DistrictHeatingCooling_Supplies_ClosedLoop_DHCSupplyHeaterCoolerStorage.mako", r"C:\Users\rka-lko\Downloads\uesgraphs_parameters_template.xlsx")

2 AUX parameter(s) not provided for supply nodes, will use Modelica defaults: dp_nominal, m_flow_nominal
C:\Users\rka-lko\AppData\Local\Temp\6\ipykernel_16216\1232783304.py:278: UserWarning: 2 AUX parameter(s) not provided for supply nodes, will use Modelica defaults: dp_nominal, m_flow_nominal
  warnings.warn(warning_msg, UserWarning)


In [12]:
def resolve_parameter_value(value, node_data, param_name, node_id):
    """Resolve parameter value - either direct value or @reference to node attribute"""
    if isinstance(value, str) and value.startswith('@'):
        # It's a reference to a node attribute
        attr_name = value[1:]  # Remove '@' prefix
        if attr_name in node_data:
            return node_data[attr_name], True  # value, was_resolved
        else:
            raise ValueError(
                f"Node {node_id}: Parameter '{param_name}' references "
                f"non-existent attribute '@{attr_name}'"
            )
    else:
        # Direct value
        return value, False
    
    
def assign_demand_parameters(uesgraph, template_path, excel_path=None, logger=None):
    """
    Assign parameters to demand nodes in the uesgraph according to the flow chart logic.
    
    This function follows the validation flow:
    1. Load and parse the template file
    2. Extract MAIN (required) and AUX (optional) parameters
    3. For each demand node individually:
       - Check MAIN parameters: in node → keep, not in node → try Excel, missing → ERROR
       - Check AUX parameters: in node → keep, not in node → try Excel, missing → WARNING
    4. Apply parameters from Excel where needed (never overwrite existing node attributes)
    
    Parameters
    ----------
    uesgraph : UESGraph
        The urban energy system graph object
    template_path : str or Path
        Path to the demand template file (.mako)
    excel_path : str or Path, optional
        Path to Excel file containing component parameters
        If None, only graph attributes are used
    logger : logging.Logger, optional
        Logger for status messages and warnings
        
    Returns
    -------
    uesgraph : UESGraph
        Updated graph with assigned parameters
    validation_report : dict
        Report containing:
        - 'success': bool, whether all MAIN parameters were found for all nodes
        - 'nodes_validated': int, number of demand nodes processed
        - 'params_from_graph': int, parameters already in graph
        - 'params_from_excel': int, parameters added from Excel
        - 'missing_main': list of (node, param) tuples for missing required parameters
        - 'missing_aux': set of parameter names missing across all nodes
        - 'errors': list of error messages
        - 'warnings': list of warning messages
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    ValueError
        If required MAIN parameters are missing for any demand node
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    validation_report = {
        'success': True,
        'nodes_validated': 0,
        'params_from_graph': 0,
        'params_from_excel': 0,
        'missing_main': [],
        'missing_aux': set(),
        'errors': [],
        'warnings': []
    }
    
    # Step 1: Load template file
    logger.info(f"Loading template file: {template_path}")
    if not os.path.exists(template_path):
        error_msg = f"Template file not found: {template_path}"
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        validation_report['success'] = False
        raise FileNotFoundError(error_msg)
    
    # Step 2: Parse template and extract parameters
    logger.info("Parsing template to extract parameter requirements")
    try:
        main_parameters, aux_parameters = parse_template_parameters(template_path, logger)
    except Exception as e:
        error_msg = f"Failed to parse template: {e}"
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        validation_report['success'] = False
        raise
    
    logger.info(f"Template requires {len(main_parameters)} MAIN and {len(aux_parameters)} AUX parameters")
    logger.debug(f"MAIN: {main_parameters}")
    logger.debug(f"AUX: {aux_parameters}")
    
    # Step 3: Load Excel parameters if provided
    excel_params = {}
    if excel_path is not None:
        try:
            logger.info(f"Loading parameters from Excel: {excel_path}")
            excel_params = load_component_parameters(excel_path, 'Demands')
            logger.debug(f"Excel parameters loaded: {list(excel_params.keys())}")
        except Exception as e:
            warning_msg = f"Could not load Excel parameters: {e}"
            logger.warning(warning_msg)
            validation_report['warnings'].append(warning_msg)
    
    # Step 4: Find demand nodes (buildings that are NOT supply)
    network_type = uesgraph.graph.get("network_type", "heating")
    is_supply_key = f"is_supply_{network_type}"
    
    demand_nodes = []
    for node in uesgraph.nodelist_building:
        if not uesgraph.nodes[node].get(is_supply_key, False):
            demand_nodes.append(node)
    
    if not demand_nodes:
        warning_msg = f"No demand nodes found (looking for buildings with '{is_supply_key}' = False)"
        logger.warning(warning_msg)
        validation_report['warnings'].append(warning_msg)
        return uesgraph, validation_report
    
    # Step 5: Process each demand node individually
    total_nodes = len(demand_nodes)
    logger.info(f"Processing {total_nodes} demand node(s)...")
    
    for node_idx, node in enumerate(demand_nodes, 1):
        node_data = uesgraph.nodes[node]
        logger.debug(f"Processing demand node {node_idx}/{total_nodes}: {node}")
        
        # Step 5a: Check and apply MAIN parameters for this node
        for param in main_parameters:
            if param in node_data:
                # Parameter already in graph - keep it
                validation_report['params_from_graph'] += 1
                logger.debug(f"  ✓ MAIN '{param}' found in node")
            elif param in excel_params:
                # Parameter not in node, but maybe available in other node or Excel - apply it
                resolved_value, was_ref = resolve_parameter_value(
                    excel_params[param], node_data, param, node
                )
                node_data[param] = resolved_value
                validation_report['params_from_excel'] += 1
                source = f"@{excel_params[param][1:]}" if was_ref else "Excel"
                logger.debug(f"  ✓ MAIN '{param}' applied from {source}")
            else:
                # Parameter missing - ERROR!
                validation_report['missing_main'].append((node, param))
                validation_report['success'] = False
                error_msg = f"Node {node}: Missing MAIN parameter '{param}'"
                logger.error(error_msg)
                validation_report['errors'].append(error_msg)
        
        # Step 5b: Check and apply AUX parameters for this node
        for param in aux_parameters:
            if param in node_data:
                # Parameter already in graph - keep it
                validation_report['params_from_graph'] += 1
                logger.debug(f"  ✓ AUX '{param}' found in node")
            elif param in excel_params:
                # Parameter not in node, but available in Excel - apply it
                resolved_value, was_ref = resolve_parameter_value(
                    excel_params[param], node_data, param, node
                )
                node_data[param] = resolved_value
                validation_report['params_from_excel'] += 1
                source = f"@{excel_params[param][1:]}" if was_ref else "Excel"
                logger.debug(f"  ✓ AUX '{param}' applied from {source}")
            else:
                # Parameter missing - will use Modelica default (collect for summary)
                validation_report['missing_aux'].add(param)
                logger.debug(f"  ⚠ AUX '{param}' not provided - will use Modelica default")
        
        validation_report['nodes_validated'] += 1
    
    # Step 6: Report summary
    logger.info(f"✓ Processed {validation_report['nodes_validated']} demand node(s)")
    logger.info(f"  - Parameters from graph: {validation_report['params_from_graph']}")
    logger.info(f"  - Parameters from Excel: {validation_report['params_from_excel']}")
    
    # Summary of missing AUX parameters (if any)
    if validation_report['missing_aux']:
        missing_count = len(validation_report['missing_aux'])
        summary_msg = (
            f"⚠ {missing_count} AUX parameter(s) not provided, will use Modelica defaults:\n"
            f"   {', '.join(sorted(validation_report['missing_aux']))}"
        )
        logger.warning(summary_msg)
        validation_report['warnings'].append(summary_msg)
    
    # Check if validation was successful
    if not validation_report['success']:
        error_count = len(validation_report['missing_main'])
        error_msg = (
            f"Validation FAILED: {error_count} missing MAIN parameter(s)\n"
            f"Missing parameters per node:\n"
        )
        for node, param in validation_report['missing_main']:
            error_msg += f"  - Node {node}: '{param}'\n"
        error_msg += (
            f"\nFix suggestions:\n"
            f"  → If parameter varies per node: add to uesgraph node attributes\n"
            f"  → If parameter is same for all demand nodes: add to Excel sheet 'Demands'"
        )
        logger.error(error_msg)
        validation_report['errors'].append(error_msg)
        raise ValueError(error_msg)
    else:
        logger.info("✓ All MAIN parameters successfully validated and applied")
    
    return uesgraph, validation_report

In [15]:
for node in graph.nodelist_building:
    graph.nodes[node]['heatDemand_max'] = 1000.0

In [20]:
assign_demand_parameters(graph, r"E:\rka_lko\git\uesgraphs\uesgraphs\data\templates\network\demand\AixLib_Fluid_DistrictHeatingCooling_Demands_ClosedLoop_DHCSubstationHeatPumpChiller.mako", r"E:\rka_lko\git\uesgraphs\uesgraphs\data\uesgraphs_parameters_template.xlsx")

3 AUX parameter(s) not provided for demand nodes, will use Modelica defaults: T_cooSecSet, T_heaSecSet, sta_default
C:\Users\rka-lko\AppData\Local\Temp\6\ipykernel_16216\1232783304.py:278: UserWarning: 3 AUX parameter(s) not provided for demand nodes, will use Modelica defaults: T_cooSecSet, T_heaSecSet, sta_default
  warnings.warn(warning_msg, UserWarning)


<uesgraphs.UESGraph object>

In [17]:
graph.nodes(data=True)

NodeDataView({1002: {'node_type': 'network_heating', 'network_id': 'network_id', 'position': <POINT (63.438 52.655)>, 'name': 1002}, 1004: {'node_type': 'network_heating', 'network_id': 'network_id', 'position': <POINT (81.811 121.298)>, 'name': 1004}, 1006: {'node_type': 'network_heating', 'network_id': 'network_id', 'position': <POINT (100.329 135.194)>, 'name': 1006}, 1007: {'node_type': 'network_heating', 'network_id': 'network_id', 'position': <POINT (27.89 81.17)>, 'name': 1007}, 1010: {'node_type': 'network_heating', 'network_id': 'network_id', 'position': <POINT (12.694 15.87)>, 'name': 1010}, 1014: {'node_type': 'network_heating', 'network_id': 'network_id', 'position': <POINT (95.28 77.03)>, 'name': 1014}, 1018: {'node_type': 'network_heating', 'network_id': 'network_id', 'position': <POINT (-4.978 56.552)>, 'name': 1018}, 1022: {'node_type': 'network_heating', 'network_id': 'network_id', 'position': <POINT (128.65 102.695)>, 'name': 1022}, 1025: {'name': 'supply1', 'node_typ

In [19]:
"""
Parameter assignment functions for uesgraph components (pipes, supply, demand).

Balanced approach:
- Clean helper functions for maintainability
- Optional logger for debugging and status messages
- Simple return value (only uesgraph)
- Exceptions for errors, warnings for non-critical issues
- Support @ references for linking Excel parameters to node/edge attributes
"""

import os
import logging
import warnings


# ============================================================================
# HELPER FUNCTIONS (shared by all assignment functions)
# ============================================================================

def resolve_parameter_value(value, component_data, param_name, component_id):
    """
    Resolve parameter value - either direct value or @reference to component attribute.
    
    Parameters
    ----------
    value : any
        The parameter value from Excel (can be direct value or @reference)
    component_data : dict
        The component's data dictionary (node or edge attributes)
    param_name : str
        Name of the parameter being resolved
    component_id : str or tuple
        Identifier of the component (node id or edge tuple)
        
    Returns
    -------
    resolved_value : any
        The resolved parameter value
        
    Raises
    ------
    ValueError
        If referenced attribute does not exist in component_data
    """
    if isinstance(value, str) and value.startswith('@'):
        # It's a reference to a component attribute
        attr_name = value[1:]  # Remove '@' prefix
        if attr_name in component_data:
            return component_data[attr_name]
        else:
            raise ValueError(
                f"Component {component_id}: Parameter '{param_name}' references "
                f"non-existent attribute '@{attr_name}'"
            )
    else:
        # Direct value
        return value


def _load_template(template_path, logger):
    """
    Load and parse template file to extract parameter requirements.
    
    Parameters
    ----------
    template_path : str or Path
        Path to template file
    logger : logging.Logger
        Logger instance
        
    Returns
    -------
    main_parameters : list
        List of MAIN parameter names
    aux_parameters : list
        List of AUX parameter names
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    Exception
        If template parsing fails
    """
    logger.info(f"Loading template file: {template_path}")
    
    if not os.path.exists(template_path):
        error_msg = f"Template file not found: {template_path}"
        logger.error(error_msg)
        raise FileNotFoundError(error_msg)
    
    logger.info("Parsing template to extract parameter requirements")
    try:
        main_parameters, aux_parameters = parse_template_parameters(template_path, logger)
    except Exception as e:
        error_msg = f"Failed to parse template: {e}"
        logger.error(error_msg)
        raise
    
    logger.info(f"Template requires {len(main_parameters)} MAIN and {len(aux_parameters)} AUX parameters")
    logger.debug(f"MAIN: {main_parameters}")
    logger.debug(f"AUX: {aux_parameters}")
    
    return main_parameters, aux_parameters


def _load_excel(excel_path, excel_sheet_name, logger):
    """
    Load parameters from Excel file.
    
    Parameters
    ----------
    excel_path : str or Path or None
        Path to Excel file (optional)
    excel_sheet_name : str
        Name of the Excel sheet to load
    logger : logging.Logger
        Logger instance
        
    Returns
    -------
    excel_params : dict
        Dictionary of parameters from Excel (empty dict if excel_path is None)
    """
    excel_params = {}
    
    if excel_path is not None:
        try:
            logger.info(f"Loading parameters from Excel: {excel_path}")
            excel_params = load_component_parameters(excel_path, excel_sheet_name)
            logger.debug(f"Excel parameters loaded: {list(excel_params.keys())}")
        except Exception as e:
            warning_msg = f"Could not load Excel parameters: {e}"
            logger.warning(warning_msg)
            warnings.warn(warning_msg, UserWarning)
    else:
        logger.info("No Excel file provided, using only graph attributes")
    
    return excel_params


def _process_component_parameters(component_id, component_data, main_parameters, 
                                  aux_parameters, excel_params, logger):
    """
    Process MAIN and AUX parameters for a single component (node or edge).
    
    This is the core validation logic shared by all three assignment functions.
    
    Parameters
    ----------
    component_id : str or tuple
        Identifier of the component (node id or edge tuple)
    component_data : dict
        The component's data dictionary
    main_parameters : list
        List of required MAIN parameter names
    aux_parameters : list
        List of optional AUX parameter names
    excel_params : dict
        Dictionary of parameters from Excel
    logger : logging.Logger
        Logger for debug messages
        
    Returns
    -------
    missing_main : list
        List of missing MAIN parameters for this component
    missing_aux : set
        Set of missing AUX parameters for this component
    stats : dict
        Statistics dict with 'from_graph' and 'from_excel' counts
    """
    missing_main = []
    missing_aux = set()
    stats = {'from_graph': 0, 'from_excel': 0}
    
    # Process MAIN parameters
    for param in main_parameters:
        if param in component_data:
            # Parameter already in component - keep it
            stats['from_graph'] += 1
            logger.debug(f"  ✓ MAIN '{param}' found in component")
        elif param in excel_params:
            # Parameter not in component, but available in Excel - apply it
            try:
                resolved_value = resolve_parameter_value(
                    excel_params[param], component_data, param, component_id
                )
                component_data[param] = resolved_value
                stats['from_excel'] += 1
                source = f"@{excel_params[param][1:]}" if isinstance(excel_params[param], str) and excel_params[param].startswith('@') else "Excel"
                logger.debug(f"  ✓ MAIN '{param}' applied from {source}")
            except ValueError as e:
                # Reference resolution failed
                missing_main.append(param)
                logger.error(f"  ✗ MAIN '{param}': {e}")
        else:
            # Parameter missing - ERROR!
            missing_main.append(param)
            logger.error(f"  ✗ MAIN '{param}' not found")
    
    # Process AUX parameters
    for param in aux_parameters:
        if param in component_data:
            # Parameter already in component - keep it
            stats['from_graph'] += 1
            logger.debug(f"  ✓ AUX '{param}' found in component")
        elif param in excel_params:
            # Parameter not in component, but available in Excel - apply it
            try:
                resolved_value = resolve_parameter_value(
                    excel_params[param], component_data, param, component_id
                )
                component_data[param] = resolved_value
                stats['from_excel'] += 1
                source = f"@{excel_params[param][1:]}" if isinstance(excel_params[param], str) and excel_params[param].startswith('@') else "Excel"
                logger.debug(f"  ✓ AUX '{param}' applied from {source}")
            except ValueError as e:
                # Reference resolution failed - treat as missing AUX
                missing_aux.add(param)
                logger.warning(f"  ⚠ AUX '{param}' reference failed: {e}")
        else:
            # Parameter missing - will use Modelica default
            missing_aux.add(param)
            logger.debug(f"  ⚠ AUX '{param}' not provided - will use Modelica default")
    
    return missing_main, missing_aux, stats


def _aggregate_statistics(all_stats_list):
    """Aggregate statistics from multiple components."""
    total_stats = {'from_graph': 0, 'from_excel': 0}
    for stats in all_stats_list:
        total_stats['from_graph'] += stats['from_graph']
        total_stats['from_excel'] += stats['from_excel']
    return total_stats


def _check_and_report_results(component_type, component_count, all_missing_main, 
                              all_missing_aux, total_stats, logger):
    """
    Check if validation was successful and report results.
    
    Parameters
    ----------
    component_type : str
        Type of component ('edge', 'supply node', 'demand node')
    component_count : int
        Number of components processed
    all_missing_main : list of tuples
        List of (component_id, param) tuples for missing MAIN parameters
    all_missing_aux : set
        Set of AUX parameter names missing across all components
    total_stats : dict
        Aggregated statistics
    logger : logging.Logger
        Logger instance
        
    Raises
    ------
    ValueError
        If any MAIN parameters are missing
    """
    # Report summary
    logger.info(f"✓ Processed {component_count} {component_type}(s)")
    logger.info(f"  - Parameters from graph: {total_stats['from_graph']}")
    logger.info(f"  - Parameters from Excel: {total_stats['from_excel']}")
    
    # Summary of missing AUX parameters
    if all_missing_aux:
        missing_count = len(all_missing_aux)
        warning_msg = (
            f"{missing_count} AUX parameter(s) not provided for {component_type}s, "
            f"will use Modelica defaults: {', '.join(sorted(all_missing_aux))}"
        )
        logger.warning(warning_msg)
        warnings.warn(warning_msg, UserWarning)
    
    # Check if validation was successful
    if all_missing_main:
        error_count = len(all_missing_main)
        error_msg = (
            f"Validation FAILED: {error_count} missing MAIN parameter(s)\n"
            f"Missing parameters per {component_type}:\n"
        )
        for component_id, param in all_missing_main:
            error_msg += f"  - {component_type.capitalize()} {component_id}: '{param}'\n"
        error_msg += (
            f"\nFix suggestions:\n"
            f"  → If parameter varies per {component_type}: add to uesgraph attributes\n"
            f"  → If parameter is same for all {component_type}s: add to Excel sheet"
        )
        logger.error(error_msg)
        raise ValueError(error_msg)
    else:
        logger.info("✓ All MAIN parameters successfully validated and applied")


# ============================================================================
# MAIN ASSIGNMENT FUNCTIONS
# ============================================================================

def assign_pipe_parameters(uesgraph, template_path, excel_path=None, logger=None):
    """
    Assign parameters to pipe edges in the uesgraph according to the flow chart logic.
    
    This function follows the validation flow:
    1. Load and parse the template file
    2. Extract MAIN (required) and AUX (optional) parameters
    3. Load Excel parameters if provided
    4. For each edge individually:
       - Check MAIN parameters: in edge → keep, not in edge → try Excel, missing → ERROR
       - Check AUX parameters: in edge → keep, not in edge → try Excel, missing → WARNING
    5. Apply parameters from Excel where needed (never overwrite existing edge attributes)
    6. Support @ references: Excel values starting with @ are resolved to edge attributes
    
    Parameters
    ----------
    uesgraph : UESGraph
        The urban energy system graph object (modified in-place)
    template_path : str or Path
        Path to the pipe template file (.mako)
    excel_path : str or Path, optional
        Path to Excel file containing component parameters
        If None, only graph attributes are used
    logger : logging.Logger, optional
        Logger for status messages and warnings
        If None, creates a default logger
        
    Returns
    -------
    uesgraph : UESGraph
        The updated graph object (same as input, modified in-place)
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    ValueError
        If required MAIN parameters are missing for any edge
        
    Warns
    -----
    UserWarning
        If optional AUX parameters are missing (will use Modelica defaults)
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    # Step 1: Load template file
    main_parameters, aux_parameters = _load_template(template_path, logger)
    
    # Step 2: Load Excel parameters if provided
    excel_params = _load_excel(excel_path, 'Pipes', logger)
    
    # Step 3: Process each edge individually
    total_edges = len(list(uesgraph.edges()))
    logger.info(f"Processing {total_edges} edge(s)...")
    
    all_missing_main = []
    all_missing_aux = set()
    all_stats = []
    
    for edge_idx, edge in enumerate(uesgraph.edges(), 1):
        edge_data = uesgraph.edges[edge]
        logger.debug(f"Processing edge {edge_idx}/{total_edges}: {edge}")
        
        missing_main, missing_aux, stats = _process_component_parameters(
            edge, edge_data, main_parameters, aux_parameters, 
            excel_params, logger
        )
        
        # Collect results
        all_missing_main.extend([(edge, param) for param in missing_main])
        all_missing_aux.update(missing_aux)
        all_stats.append(stats)
    
    # Step 4: Aggregate and report results
    total_stats = _aggregate_statistics(all_stats)
    _check_and_report_results(
        'edge', total_edges, all_missing_main, all_missing_aux, 
        total_stats, logger
    )
    
    return uesgraph


def assign_supply_parameters(uesgraph, template_path, excel_path=None, logger=None):
    """
    Assign parameters to supply nodes in the uesgraph according to the flow chart logic.
    
    This function follows the validation flow:
    1. Load and parse the template file
    2. Extract MAIN (required) and AUX (optional) parameters
    3. Load Excel parameters if provided
    4. For each supply node individually:
       - Check MAIN parameters: in node → keep, not in node → try Excel, missing → ERROR
       - Check AUX parameters: in node → keep, not in node → try Excel, missing → WARNING
    5. Apply parameters from Excel where needed (never overwrite existing node attributes)
    6. Support @ references: Excel values starting with @ are resolved to node attributes
    
    Parameters
    ----------
    uesgraph : UESGraph
        The urban energy system graph object (modified in-place)
    template_path : str or Path
        Path to the supply template file (.mako)
    excel_path : str or Path, optional
        Path to Excel file containing component parameters
        If None, only graph attributes are used
    logger : logging.Logger, optional
        Logger for status messages and warnings
        If None, creates a default logger
        
    Returns
    -------
    uesgraph : UESGraph
        The updated graph object (same as input, modified in-place)
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    ValueError
        If required MAIN parameters are missing for any supply node
        
    Warns
    -----
    UserWarning
        If optional AUX parameters are missing (will use Modelica defaults)
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    # Step 1: Load template file
    main_parameters, aux_parameters = _load_template(template_path, logger)
    
    # Step 2: Load Excel parameters if provided
    excel_params = _load_excel(excel_path, 'Supply', logger)
    
    # Step 3: Find supply nodes
    network_type = uesgraph.graph.get("network_type", "heating")
    is_supply_key = f"is_supply_{network_type}"
    
    supply_nodes = [
        node for node in uesgraph.nodelist_building
        if uesgraph.nodes[node].get(is_supply_key, False)
    ]
    
    if not supply_nodes:
        warning_msg = f"No supply nodes found (looking for '{is_supply_key}' = True)"
        logger.warning(warning_msg)
        warnings.warn(warning_msg, UserWarning)
        return uesgraph
    
    # Step 4: Process each supply node individually
    total_nodes = len(supply_nodes)
    logger.info(f"Processing {total_nodes} supply node(s)...")
    
    all_missing_main = []
    all_missing_aux = set()
    all_stats = []
    
    for node_idx, node in enumerate(supply_nodes, 1):
        node_data = uesgraph.nodes[node]
        logger.debug(f"Processing supply node {node_idx}/{total_nodes}: {node}")
        
        missing_main, missing_aux, stats = _process_component_parameters(
            node, node_data, main_parameters, aux_parameters,
            excel_params, logger
        )
        
        # Collect results
        all_missing_main.extend([(node, param) for param in missing_main])
        all_missing_aux.update(missing_aux)
        all_stats.append(stats)
    
    # Step 5: Aggregate and report results
    total_stats = _aggregate_statistics(all_stats)
    _check_and_report_results(
        'supply node', total_nodes, all_missing_main, all_missing_aux,
        total_stats, logger
    )
    
    return uesgraph


def assign_demand_parameters(uesgraph, template_path, excel_path=None, logger=None):
    """
    Assign parameters to demand nodes in the uesgraph according to the flow chart logic.
    
    This function follows the validation flow:
    1. Load and parse the template file
    2. Extract MAIN (required) and AUX (optional) parameters
    3. Load Excel parameters if provided
    4. For each demand node individually:
       - Check MAIN parameters: in node → keep, not in node → try Excel, missing → ERROR
       - Check AUX parameters: in node → keep, not in node → try Excel, missing → WARNING
    5. Apply parameters from Excel where needed (never overwrite existing node attributes)
    6. Support @ references: Excel values starting with @ are resolved to node attributes
    
    Parameters
    ----------
    uesgraph : UESGraph
        The urban energy system graph object (modified in-place)
    template_path : str or Path
        Path to the demand template file (.mako)
    excel_path : str or Path, optional
        Path to Excel file containing component parameters
        If None, only graph attributes are used
    logger : logging.Logger, optional
        Logger for status messages and warnings
        If None, creates a default logger
        
    Returns
    -------
    uesgraph : UESGraph
        The updated graph object (same as input, modified in-place)
        
    Raises
    ------
    FileNotFoundError
        If template file not found
    ValueError
        If required MAIN parameters are missing for any demand node
        
    Warns
    -----
    UserWarning
        If optional AUX parameters are missing (will use Modelica defaults)
    """
    if logger is None:
        logger = logging.getLogger(__name__)
    
    # Step 1: Load template file
    main_parameters, aux_parameters = _load_template(template_path, logger)
    
    # Step 2: Load Excel parameters if provided
    excel_params = _load_excel(excel_path, 'Demands', logger)
    
    # Step 3: Find demand nodes (buildings that are NOT supply)
    network_type = uesgraph.graph.get("network_type", "heating")
    is_supply_key = f"is_supply_{network_type}"
    
    demand_nodes = [
        node for node in uesgraph.nodelist_building
        if not uesgraph.nodes[node].get(is_supply_key, False)
    ]
    
    if not demand_nodes:
        warning_msg = f"No demand nodes found (looking for buildings with '{is_supply_key}' = False)"
        logger.warning(warning_msg)
        warnings.warn(warning_msg, UserWarning)
        return uesgraph
    
    # Step 4: Process each demand node individually
    total_nodes = len(demand_nodes)
    logger.info(f"Processing {total_nodes} demand node(s)...")
    
    all_missing_main = []
    all_missing_aux = set()
    all_stats = []
    
    for node_idx, node in enumerate(demand_nodes, 1):
        node_data = uesgraph.nodes[node]
        logger.debug(f"Processing demand node {node_idx}/{total_nodes}: {node}")
        
        missing_main, missing_aux, stats = _process_component_parameters(
            node, node_data, main_parameters, aux_parameters,
            excel_params, logger
        )
        
        # Collect results
        all_missing_main.extend([(node, param) for param in missing_main])
        all_missing_aux.update(missing_aux)
        all_stats.append(stats)
    
    # Step 5: Aggregate and report results
    total_stats = _aggregate_statistics(all_stats)
    _check_and_report_results(
        'demand node', total_nodes, all_missing_main, all_missing_aux,
        total_stats, logger
    )
    
    return uesgraph